In [4]:
# !cd venv/lib/python3.9/site-packages/pyspark/jars && wget https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/1.2.20.1043/RedshiftJDBC42-no-awssdk-1.2.20.1043.jar  /Users/giho.han/PycharmProjects/pyspark_ex/venv/lib/python3.9/site-packages/pyspark/jars
!cd venv/lib/python3.9/site-packages/pyspark/jars && wget https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/2.1.0.14/redshift-jdbc42-2.1.0.14.jar

--2023-07-05 03:01:03--  https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/2.1.0.14/redshift-jdbc42-2.1.0.14.jar
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.146.197, 52.217.196.80, 54.231.133.56, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.146.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1048241 (1024K) [binary/octet-stream]
Saving to: ‘redshift-jdbc42-2.1.0.14.jar’

redshift-jdbc42-2.1 100%[===================>]   1024K  1007KB/s    in 1.0s    

2023-07-05 03:01:05 (1007 KB/s) - ‘redshift-jdbc42-2.1.0.14.jar’ saved [1048241/1048241]



In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

conf = SparkConf()
conf.set('spark.app.name','pyspark-redshift')
conf.set('spark.master','local[*]')

spark = SparkSession.builder\
    .config(conf=conf)\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/07/05 17:09:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.conf.get('spark.jars')

Py4JJavaError: An error occurred while calling o26.get.
: java.util.NoSuchElementException: spark.jars
	at org.apache.spark.sql.errors.QueryExecutionErrors$.noSuchElementExceptionError(QueryExecutionErrors.scala:1678)
	at org.apache.spark.sql.internal.SQLConf.$anonfun$getConfString$3(SQLConf.scala:4577)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.internal.SQLConf.getConfString(SQLConf.scala:4577)
	at org.apache.spark.sql.RuntimeConfig.get(RuntimeConfig.scala:72)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:568)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gatewa

In [4]:
df_user_session_channel = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", "jdbc:redshift://learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com:5439/dev?user=guest&password=Guest1234") \
    .option("dbtable", "raw_data.user_session_channel") \
    .load()

In [5]:
df_session_timestamp = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", "jdbc:redshift://learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com:5439/dev?user=guest&password=Guest1234") \
    .option("dbtable", "raw_data.session_timestamp") \
    .load()


In [6]:
df_user_session_channel.rdd.getNumPartitions()

1

In [7]:
df_user_session_channel.printSchema()

root
 |-- userid: integer (nullable = true)
 |-- sessionid: string (nullable = true)
 |-- channel: string (nullable = true)



In [8]:
df_user_session_channel.rdd.getNumPartitions()

1

In [10]:
df_session_timestamp.printSchema()


root
 |-- sessionid: string (nullable = true)
 |-- ts: timestamp (nullable = true)



In [11]:
df_session_timestamp.rdd.getNumPartitions()


1

In [12]:
join_expr = df_user_session_channel.sessionid == df_session_timestamp.sessionid
session_df = df_user_session_channel.join(df_session_timestamp, join_expr, "inner")

In [13]:
session_df.show()

+------+--------------------+---------+--------------------+--------------------+
|userid|           sessionid|  channel|           sessionid|                  ts|
+------+--------------------+---------+--------------------+--------------------+
|   243|0226aa5193c66d990...|   Google|0226aa5193c66d990...|2019-07-01 23:04:...|
|  2776|029bf49b584c641f0...| Facebook|029bf49b584c641f0...|2019-11-11 20:37:...|
|   939|02b8d6c2775b756de...|   Google|02b8d6c2775b756de...|2019-09-01 15:29:...|
|   771|02f6364bf207d6837...|    Naver|02f6364bf207d6837...|2019-09-07 18:24:...|
|  2139|039cc47f960921e3c...|    Naver|039cc47f960921e3c...| 2019-08-01 18:19:34|
|  1011|042152452a4794754...|Instagram|042152452a4794754...|2019-11-14 17:21:...|
|  1387|05364757c1af38444...|  Organic|05364757c1af38444...| 2019-11-25 20:18:29|
|  1501|0135456d6a3c1051f...|   Google|0135456d6a3c1051f...|2019-09-24 14:49:...|
|   876|01a416a7e28d0d229...| Facebook|01a416a7e28d0d229...|2019-05-26 14:23:...|
|  2133|02ea66ee

In [20]:


session_df = df_user_session_channel.join(df_session_timestamp,join_expr,"inner").select("userid",df_user_session_channel.sessionid,"channel","ts")




In [21]:
session_df.show()

+------+--------------------+---------+--------------------+
|userid|           sessionid|  channel|                  ts|
+------+--------------------+---------+--------------------+
|  1501|0135456d6a3c1051f...|   Google|2019-09-24 14:49:...|
|   876|01a416a7e28d0d229...| Facebook|2019-05-26 14:23:...|
|  2776|029bf49b584c641f0...| Facebook|2019-11-11 20:37:...|
|  2133|02ea66ee24d57e285...|  Organic| 2019-10-28 22:50:40|
|  2139|039cc47f960921e3c...|    Naver| 2019-08-01 18:19:34|
|   599|03ce0331cbd983f16...|  Organic|2019-07-18 13:49:...|
|  1011|042152452a4794754...|Instagram|2019-11-14 17:21:...|
|  1387|05364757c1af38444...|  Organic| 2019-11-25 20:18:29|
|  1789|059e95ce5b8e879e2...|Instagram|2019-09-12 10:47:...|
|  1251|06a9e26975cd3d805...|  Youtube|2019-10-16 14:46:...|
|   264|0765933456f074d2c...|  Youtube|2019-05-05 17:55:...|
|  1051|08aedba65ce091296...|    Naver|2019-08-09 17:30:...|
|  1027|09eca040ba5cd8dfb...|   Google|2019-06-18 14:47:...|
|   277|0bf0b26798f42e0c

In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()



In [22]:
from pyspark.sql.functions import date_format,asc,countDistinct

session_df.withColumn('month',date_format('ts','yyyy-MM')).groupby('month').\
    agg(countDistinct("userid").alias('mau')).sort(asc('month')).show()

+-------+---+
|  month|mau|
+-------+---+
|2019-05|281|
|2019-06|459|
|2019-07|623|
|2019-08|662|
|2019-09|639|
|2019-10|763|
|2019-11|721|
+-------+---+



In [23]:
df_user_session_channel.createOrReplaceTempView('user_session_channel')


In [24]:
df_session_timestamp.createOrReplaceTempView('session_timestamp')

In [25]:
channel_count_df = spark.sql("""
SELECT channel, count(distinct userId) uniqueUsers
FROM session_timestamp st
JOIN user_session_channel usc ON st.sessionID = usc.sessionID
GROUP BY 1
ORDER BY 1
""")

In [26]:
channel_count_df.show()

+---------+-----------+
|  channel|uniqueUsers|
+---------+-----------+
| Facebook|        889|
|   Google|        893|
|Instagram|        895|
|    Naver|        882|
|  Organic|        895|
|  Youtube|        889|
+---------+-----------+



In [27]:
mau_df = spark.sql("""
SELECT
LEFT(A.ts, 7) AS month,
COUNT(DISTINCT B.userid) AS mau
FROM session_timestamp A
JOIN user_session_channel B ON A.sessionid = B.sessionid
GROUP BY 1
ORDER BY 1 DESc
""")

In [36]:
mau_df.orderBy('mau').show()

+-------+---+
|  month|mau|
+-------+---+
|2019-05|281|
|2019-06|459|
|2019-07|623|
|2019-09|639|
|2019-08|662|
|2019-11|721|
|2019-10|763|
+-------+---+



In [37]:
name_gneder = spark.read.format('csv').csv('name_gender.csv')

DataFrame[_c0: string, _c1: string]